In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import requests
from io import BytesIO
import os
import zipfile
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from selenium.webdriver.chrome.options import Options
import urllib.request
import shutil


download_path = '/Users/jonathanoh/Desktop/down'
script_dir = '/Users/jonathanoh/Desktop/script/crawling'
s = Service('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# script_dir = "/Users/jonathanoh/Desktop/contents/exiftoexcel/스크립트/crawling"
# download_path = "/Users/jonathanoh/Desktop/contents/exiftoexcel/temp_image_zip_treated"
# s = Service('/Users/jonathanoh/Desktop/contents/exiftoexcel/스크립트/chromedriver-mac-arm64/chromedriver')





def combine_csv_files_remove_duplicates(directory_path):
    combined_df = pd.DataFrame()

    # List to store individual dataframes
    dfs = []

    # Iterate through each file in the directory
    for file in os.listdir(directory_path):
        
        if file.endswith('.csv'):
            file_path = os.path.join(directory_path, file)
            # Read each CSV file
            df = pd.read_csv(file_path)
            while len(df.columns)>2:
                del df[df.columns[-1]]
            # Append the dataframe to the list
            dfs.append(df)
            
#     print(dfs)
    # Combine all dataframes in the list
    combined_df = pd.concat(dfs, ignore_index=True)
    if dfs:
        combined_df.columns=['url', 'filename']
    
    for file in os.listdir(directory_path):
        if file.endswith('.csv') :
            os.remove(os.path.join(directory_path, file))
    csv_file = os.path.join(directory_path, "url_data.csv")

    
    combined_df.to_csv(csv_file, index=False)
    
    # Remove duplicates
    combined_df.drop_duplicates(inplace=True)
    
#     for file in os.listdir(directory_path):
#         if file.endswith('.csv'):
            
    return combined_df



def zip_folder(folder_path, zip_name):
    # 폴더를 ZIP 파일로 압축
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), 
                           os.path.relpath(os.path.join(root, file), 
                                           os.path.join(folder_path, '..')))

def compress_and_delete_folders(base_path):
    for item in os.listdir(base_path):
        item_path = os.path.join(base_path, item)
        if os.path.isdir(item_path):
            zip_name = item_path + '.zip'
            zip_folder(item_path, zip_name)
            shutil.rmtree(item_path)
            print(f"Compressed and removed folder: {item}")



def login_to_website(driver, username, password):
    driver.get('https://arca.live/u/login?goto=%2Fb%2Faiartreal')

    username_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'username'))
    )
    username_input.send_keys(username)

    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'submitBtn'))
    )
    login_button.click()

    password_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'password'))
    )
    password_input.send_keys(password)
    login_button.click()


# def fetch_post_list(driver):
#     try:
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
#         )

#         post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")

#         posts = []
#         for post_element in post_elements:
#             href = post_element.get_attribute('href')
#             try:
#                 title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
#                 title = title_element.text
#             except Exception:
#                 title = "제목을 찾을 수 없음"
#             posts.append((href, title))

#         return posts

#     except Exception as e:
#         print(f"에러 발생: {e}")
#         return []

def fetch_post_list(driver):
    try:
        WebDriverWait(driver, 2).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
        )

        post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")
        posts = []

        for post_element in post_elements:
            href = post_element.get_attribute('href')
            title = "제목을 찾을 수 없음"
            user = "사용자를 찾을 수 없음"

            try:
                title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
                title = title_element.text if title_element else title
            except Exception:
                pass  # 제목을 찾을 수 없는 경우 기본값 사용

            try:
                user_element = post_element.find_element(By.CSS_SELECTOR, 'span[data-filter]')
                user = user_element.text if user_element else user
            except Exception:
                pass  # 사용자를 찾을 수 없는 경우 기본값 사용

            posts.append((href, title, user))

        return posts

    except Exception as e:
        print(f"에러 발생: {e}")
        return []


# def compress_images_to_zip(download_path, post_href, post_user):
#     post_id = re.search(r'\d+', post_href).group()
#     zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
#     try:
#         with zipfile.ZipFile(zip_filename, 'w') as zipf:
#             for file in os.listdir(download_path):
#                 if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
#                     file_path = os.path.join(download_path, file)
#                     zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

#         # 이미지 파일 삭제
#         for file in os.listdir(download_path):
#             if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
#                 os.remove(os.path.join(download_path, file))
    
#     except Exception as e:
#         print(f"압축 중 오류 발생: {e}")


# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     try:
#         # 페이지의 모든 <a> 태그 찾기
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url.endswith('orig') :
#             # 이미지 URL이 유효한지 확인
#                 clean_url = image_url.split('?')[0]  # 쿼리 매개변수 제거
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))  # 고유한 파일명 생성
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")
                
#                 # 이미지 다운로드
#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")


def get_file_extension(url):
    # URL에서 파일 확장자를 추출합니다.
    path = url.split('?')[0]
    extension = path.split('.')[-1]
    if '/' in extension or len(extension) > 5:
        extension = 'jpg'  # 예외 상황에 대한 기본 확장자 설정
    return extension


# def process_images(driver, post_list, download_path):
#     for href, title, user in post_list:
#         if title != "제목을 찾을 수 없음":
#             print(title)
#             download_images_from_links(driver, href, download_path, title)
#             compress_images_to_zip(download_path,href,user)
    
#     print("_____finished______")

# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     failed_downloads = []
#     try:
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url and image_url.endswith('orig'):
#                 clean_url = image_url.split('?')[0]
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#                     if os.path.getsize(file_path) < 1024:  # 가정: 파일 크기가 1KB 미만인 경우 비정상
#                         failed_downloads.append((file_name, image_url))
#                 else:
#                     failed_downloads.append((file_name, image_url))
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")
    
#     return failed_downloads


def download_images_from_links(driver, post_url, download_path, title):
    driver.get(post_url)
    time.sleep(2)  # 페이지 로딩 대기
    i = 0

    failed_downloads = []
    try:
        links = driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            image_url = link.get_attribute('href')
            if image_url and image_url.endswith('orig'):
                i+=1
                clean_url = image_url.split('?')[0]
                file_extension = get_file_extension(clean_url)
                title = title.replace("/", "_")
                file_name = title + '_' + str(links.index(link))
                file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

                response = requests.get(image_url, stream=True)
                if response.status_code == 200:
                    with open(file_path, 'wb') as file:
                        for chunk in response.iter_content(1024):
                            file.write(chunk)
                    if os.path.getsize(file_path) < 1024:  # 파일 크기 검사
                        failed_downloads.append((file_name, image_url))
                else:
                    failed_downloads.append((file_name, image_url))


        
            
    except Exception as e:
        print(f"다운로드 중 오류 발생: {e}")
#         if i==0:
#             return True
#         else :
#             return False  # 오류 발생시 실패로 간주

    if len(failed_downloads)==0 or i == 0 :
        return True
    else : 
        return False  
    



def compress_images_to_zip(download_path, post_href, post_user):
    post_id = re.search(r'\d+', post_href).group()
    zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
    try:
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in os.listdir(download_path):
                if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
                    file_path = os.path.join(download_path, file)
                    zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

        # 이미지 파일 삭제
        for file in os.listdir(download_path):
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
                os.remove(os.path.join(download_path, file))

        # ZIP 파일 크기 확인
        if os.path.getsize(zip_filename) <= 50:  # 50바이트 이하면 실패로 간주
            return False  # 실패 표시
        else:
            return True  # 성공 표시
    except Exception as e:
        print(f"압축 중 오류 발생: {e}")
        return False  # 오류 발생시 실패로 간주

def process_images(driver, post_list, download_path):
    
    duplicate_url = combine_csv_files_remove_duplicates(script_dir)
                      
    failed_posts = []
    i = 1
    total = len(post_list)
    success_url = []
    old_urlsx = duplicate_url.loc[:, 'url'].to_list()
    old_urls = []
    for x in old_urlsx :
        old_urls.append(x.split('?')[0])

    for href, title, user in post_list:
        if title != "제목을 찾을 수 없음":
            if not href.split('?')[0] in old_urls : 
                mm = href.split('?')[0].split('/')[-1]
                print(f"Processing: {total} 중 {i} {mm}__{title}  _  {user}")
                success = download_images_from_links(driver, href, download_path, title)
                is_successful_zip = compress_images_to_zip(download_path, href, user)
                if not is_successful_zip or not success:
                    failed_posts.append((href,title,user))
                    print(f"failed : {href}")
                else :
                    success_url.append([href.split('?')[0], title]) 
            else:
                print(f"중복 : {title}")
        i=i+1

    print("_____finished______")
    print("Failed posts:")
    for url, title, user in failed_posts:
        print(f"User: {user} Title: {title}, URL: {url}")
                      
    if success_url:
        temp = pd.DataFrame(success_url)
        temp.columns = ['url', 'filename']
        temp.to_csv(f"Arca_url_data.csv", index=False)
    else:
        print("No data available to create DataFrame")
    
    return failed_posts



    


In [23]:
# script_dir = '/Users/jonathanoh/Desktop/script/crawling'


options = Options()
options.page_load_strategy = 'eager'  # 'none'으로 설정할 수도 있습니다.
# driver = webdriver.Chrome('/path/to/chromedriver', options=options)

driver = webdriver.Chrome(service=s, options=options)
# 웹드라이버 초기화
# driver = webdriver.Chrome('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# 함수 호출 예시
login_to_website(driver, 'bigstone524', 'bigstone524!')



# 이후 스크립트...
post_list = fetch_post_list(driver)[7:]
post_list


#process_images(driver, post_list, download_path)

In [24]:
post_list = []
for i in range(5):
    driver.get(f"https://arca.live/b/aiartreal?p={i+1}")
    if i == 0:
        post_list = post_list+fetch_post_list(driver)[7:]
    else :
        post_list = post_list+fetch_post_list(driver)

post_list.reverse()
post_list

[('https://arca.live/b/aiartreal/95856848?p=5', 'SDXL 피쉬볼 헤드', 'thec'),
 ('https://arca.live/b/aiartreal/95856946?p=5', '연리지', '냥이집사'),
 ('https://arca.live/b/aiartreal/95857546?p=5', '밤에 빛나는', '강해린'),
 ('https://arca.live/b/aiartreal/95858202?p=5',
  'dalcefo_XL-PhotoBase-TB-Test 얼빠 안정화',
  'nonoman'),
 ('https://arca.live/b/aiartreal/95860815?p=5', '자기전에 한장', '목소리로일하자'),
 ('https://arca.live/b/aiartreal/95860891?p=5',
  '쇼 스테이지 모델 샷 + 양키센스 한숟가락',
  'RollnGo'),
 ('https://arca.live/b/aiartreal/95861711?p=5',
  '28 내믹스NeMix MuSS-N 얼짱각도',
  '로라테스터'),
 ('https://arca.live/b/aiartreal/95862929?p=5', 'Sex and Pleasure', '마이사카'),
 ('https://arca.live/b/aiartreal/95863186?p=5',
  'hanimix_real_v14+ 노출없이 감성 느낌만',
  'TeamA'),
 ('https://arca.live/b/aiartreal/95865625?p=5', 'hanmix_v19 테스트', 'ㅇㅇ'),
 ('https://arca.live/b/aiartreal/95866641?p=5', '무지성 와카', 'test8888'),
 ('https://arca.live/b/aiartreal/95868029?p=5',
  '로컬 SD 새로깔았더니 deepdanbooru 사라져서 안보이는데 어떻게 함?',
  '헥사'),
 ('https://arca.live/b

In [25]:
list = process_images(driver, post_list, download_path)
# list = process_images(driver, new, download_path)
# list = process_images(driver, [post_list[0]], download_path)

중복 : SDXL 피쉬볼 헤드
중복 : 연리지
중복 : 밤에 빛나는
중복 : dalcefo_XL-PhotoBase-TB-Test 얼빠 안정화
중복 : 자기전에 한장
중복 : 쇼 스테이지 모델 샷 + 양키센스 한숟가락
중복 : 28 내믹스NeMix MuSS-N 얼짱각도
중복 : Sex and Pleasure
중복 : hanimix_real_v14+ 노출없이 감성 느낌만
중복 : hanmix_v19 테스트
중복 : 무지성 와카
중복 : 로컬 SD 새로깔았더니 deepdanbooru 사라져서 안보이는데 어떻게 함?
중복 : 양치기소녀
중복 : 포토샵 - 너무 매끈한 피부에 질감입히기
중복 : 포토샵 - 얼빡 얼굴에 질감입히기
중복 : 막뽑
Processing: 225 중 17 95872292__네이티리님의 다희 - 판타지 풍경 - 찍먹  _  Gbox
Processing: 225 중 18 95872318__AI야 눈오는날 오두막 배경으로 이쁜 여자를 뽑아줘  _  고삐풀린뉴비
Processing: 225 중 20 95873177__bikini bottom only  _  showmaker11
Processing: 225 중 21 95873426__대충 다리벌리고 있는여자  _  목소리로일하자
Processing: 225 중 22 95875618__hanimix_v19A,B(nsfw) 모델  _  badabebe
Processing: 225 중 23 95878640__comfy) 손 고치는 신기술 - Mesh Graphormer Hand Fixing Controlnet  _  설로인
Processing: 225 중 24 95882336__스나이퍼 엔젤..  _  크런키z
Processing: 225 중 25 95882637__불패의 오피스 검스  _  설로인
Processing: 225 중 26 95882858__콘서트장에 온 티파  _  달의영혼
Processing: 225 중 27 95882989__어우씨 어렵네 이거 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ ' 이것좀 해"줘" '  

Processing: 225 중 97 95927000__조선 무사 복장과 미래 세계의 조합~  _  woohj70
failed : https://arca.live/b/aiartreal/95927000?p=3
Processing: 225 중 98 95927182__PurelyWholesome_v0.6 모델입니다.  _  badabebe
Processing: 225 중 99 95927448__네 발 포즈  _  옵치솜장인
다운로드 중 오류 발생: Message: no such element: No node with given id found
  (Session info: chrome=120.0.6099.199)
Stacktrace:
0   chromedriver                        0x00000001051484cc chromedriver + 4162764
1   chromedriver                        0x0000000105140654 chromedriver + 4130388
2   chromedriver                        0x0000000104d97bc0 chromedriver + 293824
3   chromedriver                        0x0000000104d800d8 chromedriver + 196824
4   chromedriver                        0x0000000104d7de84 chromedriver + 188036
5   chromedriver                        0x0000000104d7eb30 chromedriver + 191280
6   chromedriver                        0x0000000104da65c4 chromedriver + 353732
7   chromedriver                        0x0000000104d9daf0 chromedriver + 3

Processing: 225 중 125 95935650__hanimix_v19(2.7D_nsfw) 수영복 찍먹  _  carmen
다운로드 중 오류 발생: Message: no such element: No node with given id found
  (Session info: chrome=120.0.6099.199)
Stacktrace:
0   chromedriver                        0x00000001051484cc chromedriver + 4162764
1   chromedriver                        0x0000000105140654 chromedriver + 4130388
2   chromedriver                        0x0000000104d97bc0 chromedriver + 293824
3   chromedriver                        0x0000000104d800d8 chromedriver + 196824
4   chromedriver                        0x0000000104d7de84 chromedriver + 188036
5   chromedriver                        0x0000000104d7eb30 chromedriver + 191280
6   chromedriver                        0x0000000104da65c4 chromedriver + 353732
7   chromedriver                        0x0000000104d9daf0 chromedriver + 318192
8   chromedriver                        0x0000000104d9dbc8 chromedriver + 318408
9   chromedriver                        0x0000000104d9c17c chromedriver + 31

Processing: 225 중 147 95941442__NeMix Vurf 찍먹  _  포르투
Processing: 225 중 148 95941651__에어 리스  _  바코드
Processing: 225 중 149 95942198__모델 다수 찍먹] 흑염룡이 깃든 소녀  _  멍무이드아
Processing: 225 중 150 95942742__돌핀 팬츠 - 문제시 삭제  _  저기있는곰
Processing: 225 중 151 95943492__랜덤짤.  _  이노프리
Processing: 225 중 152 95943617__우주 간호사  _  설로인
다운로드 중 오류 발생: Message: no such element: No node with given id found
  (Session info: chrome=120.0.6099.199)
Stacktrace:
0   chromedriver                        0x00000001051484cc chromedriver + 4162764
1   chromedriver                        0x0000000105140654 chromedriver + 4130388
2   chromedriver                        0x0000000104d97bc0 chromedriver + 293824
3   chromedriver                        0x0000000104d800d8 chromedriver + 196824
4   chromedriver                        0x0000000104d7de84 chromedriver + 188036
5   chromedriver                        0x0000000104d7eb30 chromedriver + 191280
6   chromedriver                        0x0000000104da65c4 chromedriver + 35373

Processing: 225 중 204 95971390__헉헉 힘들다 ㅋㅋㅋ  _  머리는차갑게가슴은뜨겁게
failed : https://arca.live/b/aiartreal/95971390?p=1
Processing: 225 중 205 95972467__Deng_Deng_e_H34 부먹  _  빠다파리
Processing: 225 중 206 95972480__열심히 올려봅니다. 많은 그림들  _  강해린
Processing: 225 중 207 95972680__농구몇장  _  푸우
Processing: 225 중 208 95973038__YA~~~짤  _  현무
failed : https://arca.live/b/aiartreal/95973038?p=1
Processing: 225 중 209 95974959__animediff 한번 돌려봄  _  빠다파리
failed : https://arca.live/b/aiartreal/95974959?p=1
Processing: 225 중 210 95975190__hanimix_v19(2.7D_nsfw) / 찍먹  _  북이
Processing: 225 중 211 95975349__선생님들 오랜만입니다,  _  aiart
Processing: 225 중 212 95975757__하나 더~!!  _  aiart
failed : https://arca.live/b/aiartreal/95975757?p=1
Processing: 225 중 213 95976418__모자쓴 예쁜누나  _  목소리로일하자
Processing: 225 중 214 95976734__기타소녀 잡지모델 버전  _  달의영혼
Processing: 225 중 215 95976962__비타500 먹고 가  _  정배통프살
Processing: 225 중 216 95977346__성형수술  _  레낰
Processing: 225 중 217 95977757__풀 바디만 몇 장 더  _  강해린
Processing: 225 중 218 95978000__우주공간  

In [20]:
list

[('https://arca.live/b/aiartreal/95868029?p=1',
  '로컬 SD 새로깔았더니 deepdanbooru 사라져서 안보이는데 어떻게 함?',
  '헥사'),
 ('https://arca.live/b/aiartreal/95870754?p=1',
  '포토샵 - 너무 매끈한 피부에 질감입히기',
  '냥이집사')]

In [10]:
len(list)

14

In [28]:
list = process_images(driver, list, download_path)

Processing: 24 중 1 95889582__국내 생성 AI 영화 '저작권 첫 인정'...세계 2번째 사례  _  머리는차갑게가슴은뜨겁게
failed : https://arca.live/b/aiartreal/95889582?p=5
Processing: 24 중 2 95891051__경관 (와카 포함)  _  빠다파리
failed : https://arca.live/b/aiartreal/95891051?p=5
Processing: 24 중 3 95894529__성인 그림이 반실성인과 통합되고 영상 생김  _  머리는차갑게가슴은뜨겁게
failed : https://arca.live/b/aiartreal/95894529?p=4
Processing: 24 중 4 95896635__로봇걸  _  빠다파리
failed : https://arca.live/b/aiartreal/95896635?p=4
Processing: 24 중 5 95904302__t3_Ver10 모델입니다.  _  badabebe
failed : https://arca.live/b/aiartreal/95904302?p=4
Processing: 24 중 6 95905182__비누거품속의 소녀  _  빠다파리
failed : https://arca.live/b/aiartreal/95905182?p=4
Processing: 24 중 7 95914235__오늘도 한 수 배웠다.  _  이노프리
failed : https://arca.live/b/aiartreal/95914235?p=4
Processing: 24 중 8 95930640__[성공기] T2I 성공 ! 첫빠인가 !  _  머리는차갑게가슴은뜨겁게
failed : https://arca.live/b/aiartreal/95930640?p=3
Processing: 24 중 9 95931866__간호사  _  현무
failed : https://arca.live/b/aiartreal/95931866?p=3
Processing: 24 중 10 95943

In [27]:
list = process_images(driver, list, download_path)

Processing: 26 중 1 95889582__국내 생성 AI 영화 '저작권 첫 인정'...세계 2번째 사례  _  머리는차갑게가슴은뜨겁게
failed : https://arca.live/b/aiartreal/95889582?p=5
Processing: 26 중 2 95891051__경관 (와카 포함)  _  빠다파리
failed : https://arca.live/b/aiartreal/95891051?p=5
Processing: 26 중 3 95894529__성인 그림이 반실성인과 통합되고 영상 생김  _  머리는차갑게가슴은뜨겁게
failed : https://arca.live/b/aiartreal/95894529?p=4
Processing: 26 중 4 95896635__로봇걸  _  빠다파리
failed : https://arca.live/b/aiartreal/95896635?p=4
Processing: 26 중 5 95904302__t3_Ver10 모델입니다.  _  badabebe
failed : https://arca.live/b/aiartreal/95904302?p=4
Processing: 26 중 6 95905182__비누거품속의 소녀  _  빠다파리
failed : https://arca.live/b/aiartreal/95905182?p=4
Processing: 26 중 7 95907758__오랜만에 한장  _  oooi
Processing: 26 중 8 95912511__hanimix_v19B(nsfw) 찍먹 "무시무시한 지진 공포의 현장"입니다.  _  ghana
Processing: 26 중 9 95914235__오늘도 한 수 배웠다.  _  이노프리
failed : https://arca.live/b/aiartreal/95914235?p=4
Processing: 26 중 10 95930640__[성공기] T2I 성공 ! 첫빠인가 !  _  머리는차갑게가슴은뜨겁게
failed : https://arca.live/b/aiartreal/

In [122]:
# compress_and_delete_folders('/Users/jonathanoh/Desktop/참고자료')

Compressed and removed folder: 79349170
Compressed and removed folder: 80052491
Compressed and removed folder: 78772036
Compressed and removed folder: 79365121
Compressed and removed folder: 79373086
Compressed and removed folder: 79278716
Compressed and removed folder: 78848182
Compressed and removed folder: 77808111
Compressed and removed folder: 77848858
Compressed and removed folder: 79329897
Compressed and removed folder: 79459766
Compressed and removed folder: 79920618
Compressed and removed folder: 79460643
Compressed and removed folder: 80452299
Compressed and removed folder: 79319200
Compressed and removed folder: 79835700
Compressed and removed folder: 78002829
Compressed and removed folder: 79435365
Compressed and removed folder: 78323972
Compressed and removed folder: 78922361
Compressed and removed folder: 79190293
Compressed and removed folder: 78911379
Compressed and removed folder: 78906108
Compressed and removed folder: 79933720
Compressed and removed folder: 79181322


In [29]:
temp = []
for url, title, user in list :
    temp.append([url.split('?')[0], title])
df = pd.DataFrame(temp)
df.columns=['url', 'filename']
# script_dir = '/Users/jonathanoh/Desktop/script/crawling'
csv_fail_fiile = os.path.join(script_dir, 'fail_list.csv')
df.to_csv(csv_fail_fiile, index=False)
df = combine_csv_files_remove_duplicates(script_dir)

In [30]:
df = combine_csv_files_remove_duplicates(script_dir)

In [13]:
df

,url,filename
0,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4931442.jpeg
1,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4924345.jpeg
2,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926329.jpeg
3,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926315.jpeg
4,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926328.jpeg
...,...,...
4282,https://arca.live/b/aiartreal/95831624,Bikini cowboy_shot (GAMBLE-MIX-REAL-39)
4283,https://arca.live/b/aiartreal/95832608,근데 ai그림 여자 생식기 위치가 항상 너무 위 아님?
4284,https://arca.live/b/aiartreal/95836538,YA짤
4285,https://arca.live/b/aiartreal/95841469,ㅁㅁ하면... 스트레스 쌓이는군요...


In [11]:
k = [list[5]]
del list[5]

In [52]:
df['url'] = df['url'].apply(lambda x: x.split('?')[0])

In [53]:
df.to_csv(os.path.join(script_dir, "url_datas.csv"), index=False)

<bound method NDFrame.head of                                                     url  \
0     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
1     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
2     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
3     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
4     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
...                                                 ...   
7226             https://arca.live/b/aiartreal/95066061   
7227             https://arca.live/b/aiartreal/95053202   
7228             https://arca.live/b/aiartreal/95052445   
7229             https://arca.live/b/aiartreal/95074503   
9718             https://arca.live/b/aiartreal/95074892   

                                               filename  
0                              civitai_Year_127497.jpeg  
1                              civitai_Year_175719.jpeg  
2                              civitai_Year_295008.jpeg  
3                            